# Installations

In [2]:
!pip install -q packaging ninja
!pip install -q flash-attn --no-build-isolation
!pip install -q -U bitsandbytes transformers accelerate



Model Preparation


In [3]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    GenerationConfig
)

#Quantization configuration
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)

In [4]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1",
                                             quantization_config=bnb_config,
                                             load_in_4bit=True,
                                             device_map="auto")
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Data processing

In [5]:
import pandas as pd
import numpy as np
import pickle
REP = 5
RATING_OPTIONS_PEW = ['1. Morally unacceptable.', '2. Not a moral issue.', '3. Morally acceptable.']
RATING_OPTIONS_WVS = ['1. Never justifiable.', '2. Something in between.', '3. Always justifiable.']
RATING_OPTIONS_EXTENDED = ['1. Never justifiable.',
                           '2. Not justifiable.',
                           '3. Something in between.',
                           '4. Justifiable.',
                           '5. Always justifiable.'
                           ]

PEW_MAPPING = {'Q84A': 'using contraceptives',
               'Q84B':'getting a divorce',
               'Q84C': 'having an abortion',
               'Q84D':'homosexuality',
               'Q84E': 'drinking alcohol',
               'Q84F': 'married people having an affair',
               'Q84G': 'gambling',
               'Q84H': 'sex between unmarried adults'
               }

MAIN_PROMPT = """[INST]Pick one option for the following question: [QUESTION]
[OPTIONS]

Please only give me the number from the option and say nothing else. For example, don't say:
The answer is ...
[/INST]
"""
COUNTRIES_WVS_W7 = ['Cyprus', 'Ecuador', 'Greece', 'Indonesia', 'Jordan', 'Libya', 'Maldives', 'New Zealand', 'Nigeria', 'Philippines', 'Thailand', 'Tunisia', 'China', 'Singapore', 'Vietnam', 'Australia', 'Canada', 'Ethiopia', 'Kenya', 'Netherlands', 'Zimbabwe', 'Andorra', 'Argentina', 'Bangladesh', 'Armenia', 'Bolivia', 'Brazil', 'Myanmar', 'Chile', 'Taiwan', 'Colombia', 'Germany', 'Iran', 'Japan', 'Kazakhstan', 'South Korea', 'Kyrgyzstan', 'Lebanon', 'Mexico', 'Pakistan', 'Peru', 'Romania', 'Russia', 'Turkey', 'Ukraine', 'Egypt', 'United States of America', 'Mongolia', 'Nicaragua', 'Morocco', 'Venezuela', 'Iraq', 'Tajikistan', 'Guatemala', 'Malaysia']


PEW_COUNTRIES = ['United States', 'Czech Republic', 'South Korea', 'Canada', 'France', 'Germany', 'Spain', 'Mexico', 'Chile', 'Australia', 'Russia', 'Britain', 'Turkey', 'Greece', 'Egypt', 'Poland', 'Senegal', 'Italy', 'Brazil', 'Lebanon', 'Nigeria', 'Japan', 'Malaysia', 'Kenya', 'Indonesia', 'Uganda', 'Jordan', 'Argentina', 'Philippines', 'Tunisia', 'China', 'Pakistan', 'Ghana', 'South Africa', 'Palestinian territories', 'Israel', 'Bolivia', 'Venezuela', 'El Salvador']


MINUS = 5.5
DIVIDE = 4.5

W7_QUESTIONS = ['Q'+str(i) for i in range(177, 196)]
W7_QUESTIONS_TEXT = ['claiming government benefits to which you are not entitled',
                     'avoiding a fare on public transport',
                     'stealing property',
                     'cheating on taxes',
                     'someone accepting a bribe in the course of their duties',
                     'homosexuality',
                     'prostitution',
                     'abortion',
                     'divorce',
                     'sex before marriage',
                     'suicide',
                     'euthanasia',
                     'for a man to beat his wife',
                     'parents beating children',
                     'violence against other people',
                     'terrorism as a political, ideological or religious mean',
                     'having casual sex',
                     'political violence',
                     'death penalty']

QUESTION_WAVES = {7: (W7_QUESTIONS, W7_QUESTIONS_TEXT)}

In [6]:
#pull wvs and pew data

def get_wvs_df(wave = 7):
    wvs_df = pd.read_csv(f'/kaggle/input/kmoral/F00011356-WVS_Cross-National_Wave_7_csv_v5_0/WVS_Cross-National_Wave_7_csv_v5_0.csv')
    wvs_df_country_names = pd.read_csv('/kaggle/input/kmoral/country_names.csv')
    wvs_df_country_names = wvs_df_country_names.drop_duplicates(['B_COUNTRY'])[['B_COUNTRY','StateNme']]
    wvs_df = wvs_df.merge(wvs_df_country_names, on = 'B_COUNTRY', how = 'left')
    return wvs_df

def get_pew_moral_df():
    df = pd.read_csv('/kaggle/input/kmoral/PEW_2013_moral_small.csv')
    return df

In [7]:
def get_culture_prompting(culture, prompt_mode):
    if prompt_mode == 'in':
        return f'people in {culture}'
    else:
        pass

In [8]:
def get_universal_prompts_for_mistral():
    prompts = {}
    culture_prompt = 'people'
    question = f'Do {culture_prompt} believe that '
    
    questions = PEW_MAPPING.keys()
    for q in questions:
        question_prompt = f'{question}{PEW_MAPPING[q]} is:\n'
        main_prompt_copy = MAIN_PROMPT
        main_prompt_copy = main_prompt_copy.replace('[QUESTION]',question_prompt)
        main_prompt_copy = main_prompt_copy.replace('[OPTIONS]','\n'.join(RATING_OPTIONS_PEW))
        prompts[PEW_MAPPING[q]] = main_prompt_copy
    return prompts

In [9]:
def mort_prompts_pew_style(culture ='', prompt_mode ='in', include_atoms = False):
    '''
    :param culture: country
    :param prompt_mode:  refer to the country in the prompt
    :param include_atoms: add the single verbs with no context to the prompt list
    :return: List[Tuple(str,str,str)] Return the list of prompts. Each prompt is a tuple of
    (the prompt question, the atomic action, the context)
    '''

    d = pd.read_csv('/kaggle/input/kmoral/list.csv')
    prompts = []
    if len(culture) > 0:
        culture_prompt = get_culture_prompting(culture, prompt_mode)
    else:
        culture_prompt = 'people'
    question = f'Do {culture_prompt} believe that '

    for i, row in d.iterrows():
        aa = row['AA Gerund']
        acis = row['ACI']
        acis = str.split(acis,';')
        if include_atoms:
            acis.append('')

        for aci in acis:
            actions = get_pew_prompt_for_mort(aa, aci, question)
            prompts.append((actions, row['AA'], aci))

    return prompts

In [10]:
def get_pew_prompt_for_mort(aa, aci, question):
    
    question_prompt = f'{question}{aa} {aci} is:\n'
    main_prompt_copy = MAIN_PROMPT
    main_prompt_copy = main_prompt_copy.replace('[QUESTION]',question_prompt)
    main_prompt_copy = main_prompt_copy.replace('[OPTIONS]','\n'.join(RATING_OPTIONS_PEW))
        
    return main_prompt_copy

----------------------------------------------------------------------------------------

Ratings utils

In [11]:
def get_pew_ratings(pew_df, culture,q ):
    rating_answers = {'Morally acceptable', 'Morally unacceptable', 'Not a moral issue'}
    df = pew_df[['COUNTRY', q]]
    df = df.loc[df['COUNTRY'] == culture]
    if len(df) == 0:
        answers = {r: 0 for r in rating_answers}
    else:
        answers = {r: len(df.loc[df[q] == r]) / len(df) for r in rating_answers}
    return answers

def get_wvs_ratings(wvs_df, culture, q):
    df = wvs_df[['StateNme', q]]
    df = df.loc[df['StateNme'] == culture]
    if len(df) == 0:
        return None

    ratings = df.loc[df[q] > 0][q]
    ratings = ((ratings - MINUS) / DIVIDE).mean()
    return ratings

In [12]:
def pew_mistral_prompts_ratings(pew_df:pd.DataFrame, culture ='', prompt_mode ='in'):

    prompts = {}
    if len(culture) > 0:
        culture_prompt = get_culture_prompting(culture, prompt_mode)
    else:
        culture_prompt = 'people'
    question = f'Do {culture_prompt} believe that '

    questions = PEW_MAPPING.keys()
    for q in questions:
        rating_answers = get_pew_ratings(pew_df, culture, q)
        
        question_prompt = f'{question}{PEW_MAPPING[q]} is:\n'
        main_prompt_copy = MAIN_PROMPT
        main_prompt_copy = main_prompt_copy.replace('[QUESTION]',question_prompt)
        main_prompt_copy = main_prompt_copy.replace('[OPTIONS]','\n'.join(RATING_OPTIONS_PEW))
        
        prompts[PEW_MAPPING[q]] = (rating_answers, main_prompt_copy,
                                   rating_answers['Morally acceptable'] - rating_answers['Morally unacceptable'])
    return prompts

def wvs_mistral_prompts_ratings(wvs_df:pd.DataFrame, culture ='', prompt_mode ='in', wave = 7, extended_rating = False):

    rating = RATING_OPTIONS_EXTENDED if extended_rating else RATING_OPTIONS_WVS
    prompts = {}
    if len(culture) > 0:
        culture_prompt = get_culture_prompting(culture, prompt_mode)
    else:
        culture_prompt = 'people'
    question_start = f'Do {culture_prompt} believe that '

    questions, questions_text = QUESTION_WAVES[wave]

    for q, q_text in zip(questions, questions_text):
        rating_answers = get_wvs_ratings(wvs_df, culture, q) #is the average ratings from the countru
        
        question_prompt = f'{question_start}{q_text} is:\n'
        main_prompt_copy = MAIN_PROMPT
        main_prompt_copy = main_prompt_copy.replace('[QUESTION]',question_prompt)
        main_prompt_copy = main_prompt_copy.replace('[OPTIONS]','\n'.join(rating))
        
        prompts[q_text] = (rating_answers, main_prompt_copy)
    return prompts

In [13]:
def get_mistral_response(text:str, temperature = 0.6, max_tokens=50, top_p=1) -> str:
    gen_config = GenerationConfig(
        pad_token_id=tokenizer.pad_token_id,
        temperature=temperature,
        top_p=top_p,
        max_new_tokens=max_tokens,
        do_sample=True,
        num_beams=1
    )
    
    model_inputs = tokenizer(text, return_tensors="pt", padding=True).to("cuda")

    with torch.no_grad():
                generated_ids = model.generate(**model_inputs,
                                               generation_config=gen_config)
    #log_prob = torch.nn.functional.log_softmax(generated_ids,dim=-1)
    generated_text = tokenizer.batch_decode(generated_ids)
    
    s = generated_text[0]
    return s[s.rfind("\n")+1:s.rfind("</s>")]

In [14]:
def access_mistral(question, question_prompt, choices, country_responses, repeat_num):

    for i in range(repeat_num):
        lm_response = get_mistral_response(question_prompt)
        while lm_response not in choices:
            lm_response = get_mistral_response(question_prompt)

        lm_score = float(lm_response[0]) - 2
        country_responses[question].append(lm_score)

    question_var = np.var(country_responses[question])
    question_mean = np.mean(country_responses[question])
    return question_var, question_mean

---------------------------------------------------------------------------------------------------------------

In [24]:
def universal_mistral_pew(repeat = REP, cultures : list = None):
    pew_df = get_pew_moral_df()

    if cultures == None:
        cultures = list(pew_df['COUNTRY'].unique())
    gpt3_all = []
    responses = {}
    universal_prompts = get_universal_prompts_for_mistral()
    for question, question_prompt in universal_prompts.items():
        
        question_scores = []
        responses[question] = question_scores
        print(question_prompt)
        question_var, question_scores = access_mistral(question, question_prompt, RATING_OPTIONS_PEW,
                                                   responses, repeat)
        print(question_prompt)
        for culture in cultures:
            print(culture)
            prompts = pew_mistral_prompts_ratings(pew_df, culture)
            rating_scores = []
            ratings = prompts[question]

            pew_score = ratings[2] #rating_answers['Morally acceptable'] - rating_answers['Morally unacceptable']
            rating_scores.append(pew_score)

            row = {'country': culture, 'prompt': question_prompt, 'topic': question, 'pew_rating': ratings[0],
                   'pew_score': pew_score, 'gpt3_score_mean':question_scores, 'gpt_var':question_var,
                   'repeat' : repeat}
            gpt3_all.append(row)
        print('----------------------------------')

    df = pd.DataFrame(gpt3_all)
    df.to_csv('/kaggle/working/universal_pew_mistral.csv', index = False )

In [27]:

def universal_mistral_wvs(repeat = REP, cultures : list = None, wave = 7):

    wvs_df = get_wvs_df(wave)

    if cultures == None:
        cultures = COUNTRIES_WVS_W7
    gpt3_all = []
    responses = {}
    universal_prompts = wvs_mistral_prompts_ratings(wvs_df)
    for question, question_prompt in universal_prompts.items():
        question_scores = []
        responses[question] = question_scores
        print(question_prompt[1])
        question_var, question_scores = access_mistral(question, question_prompt[1], RATING_OPTIONS_WVS,
                                                   responses, repeat)
        #time.sleep(30)
        for culture in cultures:
            prompts = wvs_mistral_prompts_ratings(wvs_df, culture)
            rating_scores = []
            ratings = prompts[question]

            wvs_score = ratings[0]
            rating_scores.append(wvs_score)

            row = {'country': culture, 'prompt': question_prompt[1], 'topic': question,
                   'wvs_score': wvs_score, 'gpt3_score_mean':question_scores, 'gpt_var':question_var,
                   'repeat' : repeat}
            gpt3_all.append(row)

    df = pd.DataFrame(gpt3_all)
    df.to_csv(f'/kaggle/working/universal_wvs_w{wave}_mistral.csv', index = False )

compare_gpt3_pew


In [17]:
def compare_mistral_pew(repeat = REP, cultures : list = None):


    pew_df = get_pew_moral_df()
    if cultures == None:
        cultures = list(pew_df['COUNTRY'].unique())
    gpt3_all = []
    for culture in cultures:
        print(culture)
        prompts = pew_mistral_prompts_ratings(pew_df, culture)


        country_rows = []
        country_responses = {}
        for question,ratings in prompts.items():
            print(question)
            country_responses[question] = []
            pew_score = ratings[2]
            question_prompt = ratings[1]
            question_var,question_scores = access_mistral(question, question_prompt,
                                                     RATING_OPTIONS_PEW, country_responses, repeat)

            row = {'country': culture, 'prompt': question_prompt, 'topic': question, 'pew_rating': ratings[0],
                   'pew_score': pew_score, 'gpt3_score_mean':question_scores, 'gpt_var':question_var,
                   'repeat' : repeat}
            gpt3_all.append(row)
            country_rows.append(row)
            #time.sleep(30)
        print('----------------------------------------------')

    df = pd.DataFrame(gpt3_all)
    df.to_csv('/kaggle/working/pew_mistral.csv', index = False )

compare_gpt3_wvs

In [18]:
def compare_mistral_wvs(repeat = REP, cultures : list = None, wave = 7,
                     extend = False, rating_type = ''):

    wvs_df = get_wvs_df(wave)
    cultures = COUNTRIES_WVS_W7 if cultures == None else cultures
    extended_rating = True if rating_type == '_extended' else False

    gpt3_all = []

    for culture in cultures:
        print(culture)
        country_responses = {}
        if extend:
            country_responses = pickle.load(open(f'data/WVS/{culture}_wvs_w{wave}_mistral.p', 'rb'))
        prompts = wvs_mistral_prompts_ratings(wvs_df, culture, wave=wave,extended_rating=extended_rating)

        country_rows = []
        for question,ratings in prompts.items():
            print(question)
            if not extend:
                country_responses[question] = []
                repeat_num = repeat
            else:
                repeat_num = repeat - len(country_responses[question])

            wvs_score = ratings[0]
            question_prompt = ratings[1]

            question_var,question_scores= access_mistral(question, question_prompt,
                                                     RATING_OPTIONS_WVS, country_responses, repeat_num)

            row = {'country': culture, 'prompt': question_prompt, 'topic': question,
                   'wvs_score': wvs_score, 'gpt3_score_mean':question_scores, 'gpt_var':question_var,
                   'repeat' : repeat}

            gpt3_all.append(row)
            country_rows.append(row)
            #time.sleep(30)
        print("-------------------------------------------")
        pickle.dump(country_responses, open(f'/kaggle/working/{culture}_wvs_w{wave}_mistral{rating_type}.p', 'wb'))

    df = pd.DataFrame(gpt3_all)
    df.to_csv(f'/kaggle/working/wvs_w{wave}_mistral{rating_type}.csv', index = False )

compare_gpt3_prompts_mort_user_study

In [19]:
def compare_mistral_prompts_mort_user_study(repeat = REP, user_study = 'globalAMT'):

    universal_mort_prompts = mort_prompts_pew_style(include_atoms = True)
    user_df = pd.read_csv(f'/kaggle/input/kmoral/userStudy_scores_{user_study}.csv')
    list_rows = []
    responses = {}
    for question_prompt,aa, aci in universal_mort_prompts:
        print(question_prompt)
        question_scores = []
        responses[question_prompt] = question_scores

        if aci == '':
            user_score = user_df.loc[user_df.action == aa]['score']
        else:
            user_score = user_df.loc[user_df.action.str.contains(aa)].loc[user_df.action.str.contains(aci)]['score']
        if len(user_score) == 0:
            continue

        user_score = (float(list(user_score)[0]) - 0.5) / 0.5
        question_var, question_scores = access_mistral(question_prompt, question_prompt,
                                                   RATING_OPTIONS_PEW, responses, repeat)

        row = {'prompt': aa + ' ' + aci,'gpt3_score_mean':question_scores, 'gpt_var':question_var,
               'repeat' : repeat, 'user_score' : user_score}
        list_rows.append(row)
    df = pd.DataFrame(list_rows)
    df.to_csv(f'/kaggle/working/mistral_mort_userstudy_{user_study}.csv', index = False)
        #time.sleep(30)

compare_gpt3_pew

In [28]:
universal_mistral_wvs(repeat=REP)
#compare_mistral_pew(repeat = REP, cultures=PEW_COUNTRIES)
#compare_mistral_wvs(repeat=REP, cultures=COUNTRIES_WVS_W7)
#compare_mistral_prompts_mort_user_study(repeat= REP, user_study='globalAMT')

/tmp/ipykernel_47/1822658705.py:4: DtypeWarning: Columns (345,347,349,574,575,576) have mixed types. Specify dtype option on import or set low_memory=False.
  wvs_df = pd.read_csv(f'/kaggle/input/kmoral/F00011356-WVS_Cross-National_Wave_7_csv_v5_0/WVS_Cross-National_Wave_7_csv_v5_0.csv')


[INST]Pick one option for the following question: Do people believe that claiming government benefits to which you are not entitled is:

1. Never justifiable.
2. Something in between.
3. Always justifiable.

Please only give me the number from the option and say nothing else. For example, don't say:
The answer is ...
[/INST]

[INST]Pick one option for the following question: Do people believe that avoiding a fare on public transport is:

1. Never justifiable.
2. Something in between.
3. Always justifiable.

Please only give me the number from the option and say nothing else. For example, don't say:
The answer is ...
[/INST]

[INST]Pick one option for the following question: Do people believe that stealing property is:

1. Never justifiable.
2. Something in between.
3. Always justifiable.

Please only give me the number from the option and say nothing else. For example, don't say:
The answer is ...
[/INST]

[INST]Pick one option for the following question: Do people believe that cheatin

## Inference